In [1]:
import gzip
import json
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
with gzip.open('linkedin_curaleaf.json.gz', "rb") as f:
    content = json.loads(f.read())["content"]

# parse data
soup = BeautifulSoup(content, 'html.parser')

# list posts
posts = soup.findAll('div', {'class': 'occludable-update ember-view'})

In [3]:
def soupify(gz_file):
    
    # open file
    with gzip.open(gz_file, "rb") as f:
        content = json.loads(f.read())["content"]
    
    # parse data
    soup = BeautifulSoup(content, 'html.parser')
    
    # list posts
    posts = soup.findAll('div', {'class': 'occludable-update ember-view'})
    
    df = pd.DataFrame()
    
    # iterate through each post and extract metadata
    for index, x in enumerate(posts):

        try:
            
            # views (video)
            a = x.find('ul', {'class': 'social-details-social-counts ember-view'})
            try:
                views = a.findAll('button', {'class': 'social-details-social-counts__count-value t-12 t-black--light t-normal hoverable-link-text'})[-1].get_text().replace('\n', '')
            except:
                views = None
            
            # post image
            try:
                image = x.find('button', {'class': 'feed-shared-image__image-link'}).find('img').get('src')
            except:
                image = None
            
            # post
            post = x.find('div', {'class': 'feed-shared-update-v2__description-wrapper ember-view'}).get_text().replace('\n', '')

            # likes
            likes = x.find('button', {'data-control-name': 'likes_count'}).get_text().replace('\n', '')

            # date
            date = x.find('span', {'aria-hidden': 'true'}).get_text().replace('\n', '')

            # post id
            try:
                ids = x.find('div', {'class': 'feed-shared-update-v2 feed-shared-update-v2--minimal-padding relative full-height feed-shared-update-v2--e2e Elevation-2dp ember-view'}).get('data-urn').replace('\n', '')
            except:
                ids = x.find('div', {'class': 'feed-shared-update-v2 feed-shared-update-v2--minimal-padding relative full-height Elevation-2dp ember-view'}).get('data-urn').replace('\n', '')

            # number of comments
            try:
                comment = x.find('button', {'data-control-name': 'comments_count'}).get_text().replace('\n', '')
            except:
                comment = '0'
            
            # add comments
            coment = []

            try:
                ido = x.findAll('a', {'class': 'tap-target comments-post-meta__actor-link ember-view'})
                dates = x.findAll('time', {'class': 'comments-comment-item__timestamp t-12 t-black--light t-normal mr1'})
                comment_text = x.findAll('p', {'class': 'comments-comment-item__main-content feed-shared-main-content--comment t-14 t-black t-normal feed-shared-main-content ember-view'})
                username = x.findAll('span', {'class': 'comments-post-meta__name t-14 t-black t-bold'})
                username_title = x.findAll('span', {'class': 'comments-post-meta__headline t-12 t-black--light t-normal'})
                comment_like = x.findAll('div', {'class': 'comments-comment-social-bar display-flex ember-view'})
                                    
                for u,q,w,x,y,z in zip(comment_like, username_title, ido, username, dates, comment_text):
                    try:
                        a = u.find('button', {'data-control-name': "comment_like_count"}).find('span').get_text()
                    except:
                        a = '0'
                    b = q.get_text().replace('\n', '').strip()
                    d = 'https://www.linkedin.com' + w.get('href')
                    e = x.find('span').get_text()
                    f = y.get_text()
                    g = z.get_text()
                    coment.append([a,b,d,e,f,g])
                
            except:
                pass
            
            # append parsed data to dataframe
            df = df.append(pd.DataFrame([{'views': views, 'posts': post, 'likes': likes, 'date': date, 'ids': ids, 'comment': comment, 'coment': coment, 'image': image}]))
        except AttributeError:
            pass
    
    # clean data
    df = df[['ids', 'posts', 'date', 'likes', 'comment', 'views', 'coment', 'image']]
    df.columns = ['ID', 'post', 'date', 'like', 'comment', 'view', 'coment', 'image']
    df['ID'] = df.ID.str.split(':', expand = True)[3]
    df['date'] = df.date.str.split(' ', expand = True)[0]
    df['comment'] = df.comment.str.split(' ', expand = True)[0]
    df['view'] = df['view'].apply(lambda x: x.split(' ')[0] if x[-2:] == 'ws' else None)
    df.reset_index(drop=True, inplace=True)
    
    df = df.explode('coment')
    df.reset_index(drop=True, inplace=True)
    df1 = pd.DataFrame()
    for i, row in df.iterrows():
        df = pd.DataFrame([row.coment])
        df['ID'] = row['ID']
        df['post'] = row['post']
        df['image'] = row['image']
        df['date'] = row['date']
        df['like'] = row['like']
        df['comment'] = row['comment']
        df['view'] = row['view']
        df1 = df1.append(df)
        
    df1.columns = ['comment_like', 'comment_username_title','comment_bio', 'comment_username', 'comment_date', 'comment', 'post_id', 'post', 'image', 'post_date', 'post_like', 'post_comment', 'post_view']
    df1['comment_like'] = df1.comment_like.str.split(' ', expand = True)[0]
    df1 = df1[['post_id', 'post', 'post_date', 'post_like', 'post_comment', 'image', 'post_view', 'comment_username','comment_username_title', 'comment_bio', 'comment_date', 'comment', 'comment_like']]
    df1.reset_index(drop=True, inplace=True)

    return df1
        

In [4]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [5]:
a = soupify('linkedin_curaleaf.json.gz')

/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:7123: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


In [6]:
a.to_csv('final.csv', index=False)